In [ ]:
#Pull the the requirements file and packages 
!curl 

In [1]:
#install dependencies
%pip install --no-cache-dir --upgrade pip

#Install the cython package
%pip install --no-cache-dir cython

#Install all the dependencies
%pip install --no-cache-dir -r requirements.txt

#install the package using apt with packages.txt file
# %apt-get update && cat packages.txt | xargs apt-get install -y

#Install Google Protobuf
%pip install protobuf==4.21.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 547.4 kB/s eta 0:00:00m eta 0:00:010:01:01
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 122.3 kB/s eta 0:00:00m eta 0:00:010:00:01
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 470.3 kB/s eta 0:00:00 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 kB 1.9 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.

In [2]:
#Import the dependencies
import os
import nemo.collections.asr as nemo_asr
from pydub import AudioSegment
import pyaudioconvert as pac

#import the model using hugging face 
hf_model = nemo_asr.models.EncDecRNNTBPEModel.from_pretrained(model_name="mbazaNLP/Kinyarwanda_nemo_stt_conformer_model")

[NeMo W 2024-01-19 14:49:36 transformer_bpe_models:59] Could not import NeMo NLP collection which is required for speech translation model.


[NeMo I 2024-01-19 14:49:40 mixins:170] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2024-01-19 14:49:41 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/chanie/asr/manifest/train_tarred_1bk/tarred_audio_manifest.json
    sample_rate: 16000
    batch_size: 32
    shuffle: true
    num_workers: 4
    pin_memory: true
    use_start_end_token: false
    trim_silence: false
    max_duration: 16.7
    min_duration: 0.1
    is_tarred: true
    tarred_audio_filepaths: /data/chanie/asr/manifest/train_tarred_1bk/audio__OP_0..1023_CL_.tar
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    
[NeMo W 2024-01-19 14:49:41 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Valida

[NeMo I 2024-01-19 14:49:41 features:289] PADDING: 0


[NeMo W 2024-01-19 14:49:42 asr_confidence_utils:195] `method_cfg` is deprecated and will be removed in the future. Please use `measure_cfg` instead.
[NeMo W 2024-01-19 14:49:42 asr_confidence_utils:200] Re-writing `measure_cfg` with the value of `method_cfg`.
[NeMo W 2024-01-19 14:49:42 asr_confidence_utils:95] `temperature` is deprecated and will be removed in the future. Please use `alpha` instead.
[NeMo W 2024-01-19 14:49:42 asr_confidence_utils:100] Re-writing `alpha` with the value of `temperature`.
[NeMo W 2024-01-19 14:49:42 asr_confidence_utils:195] `method_cfg` is deprecated and will be removed in the future. Please use `measure_cfg` instead.
[NeMo W 2024-01-19 14:49:42 asr_confidence_utils:200] Re-writing `measure_cfg` with the value of `method_cfg`.
[NeMo W 2024-01-19 14:49:42 asr_confidence_utils:95] `temperature` is deprecated and will be removed in the future. Please use `alpha` instead.
[NeMo W 2024-01-19 14:49:42 asr_confidence_utils:100] Re-writing `alpha` with the va

[NeMo I 2024-01-19 14:49:42 save_restore_connector:249] Model EncDecCTCModelBPE was successfully restored from /Users/kayarn/.cache/huggingface/hub/models--mbazaNLP--Kinyarwanda_nemo_stt_conformer_model/snapshots/6a95dc2c0015c9fb40d90e96bec55e7566a43f2c/Kinyarwanda_nemo_stt_conformer_model.nemo.


In [ ]:
#function to transcribe the audio



class Transcriber:
    def __init__(self, audio_bytes : bytes) -> None:
        self.audio_bytes = audio_bytes

        #save the audio
        self.save_audio()

        #convert the audio file
        self.convert_wav_to_16bit_mono()

        #Transcribe
        self.transcription = self.transcribe()

    def save_audio(self):
        self.file_id = len(os.listdir('sounds/'))
        with open(f"sounds/sound-{self.file_id}.wav", "wb") as audio_file:
            audio_file.seek(0)
            audio_file.write(self.audio_bytes)


    def convert_wav_to_16bit_mono(self):
        try:
            file_path = f"sounds/sound-{self.file_id}.wav"
            pac.convert_wav_to_16bit_mono(file_path,file_path)
            return True
        except FileNotFoundError:
            return False
        
    def transcribe(self):
        try:
            file_path = f"sounds/sound-{self.file_id}.wav"
            result= hf_model.transcribe([file_path])
            return result[0]
        except FileNotFoundError:
            return "Unable to transcribe audio!"



